In [1]:
import chromadb
import openai
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
import datetime as dt

API_KEY = os.getenv('OAI_API_KEY')


In [2]:
df = pd.read_csv('./data/saigai_data.csv')
df = df.astype({'年': 'Int64', '月': 'Int64', '業種（中分類）': 'Int64', '業種（小分類）': 'Int64', '事故の型': 'Int64', '起因物（大分類）.2': 'Int64', '起因物（中分類）.2': 'Int64', '起因物（小分類）.2': 'Int64'})
df_new = df[['年号','年','月','災害状況','業種（大分類）.1']]

/var/folders/5s/lns070lx7x3_4_7qbhgzsj0w0000gn/T/ipykernel_15090/2698770361.py:1: DtypeWarning: Columns (7,14,16,17,18,22,24,26,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/saigai_data.csv')


In [19]:
df_new.head(10)

,年号,年,月,災害状況,業種（大分類）.1
0,平成,25,1,配送作業中、下り坂を急ぎ足で下っていたところ、雪で足が滑り、手をつき、転倒した。,通信業
1,平成,25,1,ホテル正面玄関を雪かきしていた際、滑り、後ろ向きに転倒。建物に片足が引っ掛かり、骨折した。,接客娯楽業
2,平成,25,1,運転所構内の通路にて、車内の内部清掃をしようと両手に荷物を持ち、徒歩で移動していた際、降雪し...,清掃・と畜業
3,平成,25,1,競馬場にて、走路整備の作業中、障害の上部の高さを一定にしようと、水糸を持ち、移動する際、体立...,清掃・と畜業
4,平成,25,1,コンドミニアムの除雪と氷を除去する作業中、スコップで氷を割った際、手を負傷した。,金融・広告業
5,平成,25,1,入浴の為、患者をストレッチャーに乗せ、方向転換をしていた際、後方の担当者が前を確認せず、動か...,保健・衛生業
6,平成,25,1,施設の玄関ポーチの屋根にて、雪下ろしを終え、梯子で降りようと足をかけた際、靴底に着いた雪で滑...,保健・衛生業
7,平成,25,1,夜間警備業務につくため、体育センターの玄関脇スロープを歩き、玄関に向う途中、雪が乗った路面に...,その他の事業
8,平成,25,1,エビ篭作業中、大波を受け、船が揺れ、転び、全身を船体に強打し、肘、膝を負傷した。,畜産・水産業
9,平成,25,1,工場内にて、清掃作業中、通常はスクリューを止めてする作業を作動した状態で行ったため、手指の先...,製造業


In [21]:
df_new.tail(10)

,年号,年,月,災害状況,業種（大分類）.1
155383,平成,29,12,バックルームで、コンテナー（縦60㎝×横45㎝×高さ45㎝、重さ1㎏）をカーゴ車に移動してい...,商業
155384,平成,29,12,下水道補修工事現場で、マンホール内に入り、上流から来る汚水を止めるため、止水プラグ（Φ200...,建設業
155385,平成,29,12,客を降車させ、駐車場内を移動中に、前方の車がブレーキ・急バックをしてきたため、衝突を回避しよ...,運輸交通業
155386,平成,29,12,客室にて、バスタブの清掃中に滑り、タイル張りの床に右膝を強打した。,その他の事業
155387,平成,29,12,当社育苗棟内にて、苗の鉢ずらし作業をしていた際、加温機のダクト（ビニールホース）に足を引っ掛...,農林業
155388,平成,29,12,配達に出て、一件目の配達先で荷台観音ドアを開けたときに、荷台上部から重量約20㎏はあると思わ...,運輸交通業
155389,平成,29,12,当社敷地内のゴミ置き場において、ゴミ分別作業中、瓶入りの袋を開けたところ、割れた瓶が入ってお...,製造業
155390,平成,29,12,精肉作業場で、ダンボールを片づけているとき、ダンボールに躓いて転倒した。,商業
155391,平成,29,12,園の門を閉めているときに、閉める勢いで門に付いている鍵が落ちてきて、左手人差指を挟み、切って...,製造業
155392,平成,29,12,利用者居室にて、利用者をベッドより車椅子へ移乗する際に、利用者が右側へバランスを崩したので、...,保健・衛生業


In [3]:
client = chromadb.Client()

In [4]:
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=API_KEY,
                model_name="text-embedding-ada-002"
            )
collection = client.create_collection(name="disaster_collection",embedding_function=openai_ef)


In [12]:
df_dic = df_new[:1000].to_dict()
docs = list(df_dic['災害状況'].values())
# df_dicから'災害状況'キーを除いた辞書を取り出すコード
original_data = {key: value for key, value in df_dic.items() if key != '災害状況'}
metadata = [
    {key: value[index] for key, value in original_data.items()}
    for index in range(len(next(iter(original_data.values()))))
]

ids = ["id{}".format(i) for i in df_dic['災害状況'].keys()]



In [13]:
collection.add(
    documents=docs,
    metadatas=metadata,
    ids=ids
)

In [14]:
results = collection.query(
    query_texts=["季節性災害"],
    n_results=10
)
results

{'ids': [['id128',
   'id432',
   'id436',
   'id615',
   'id191',
   'id116',
   'id286',
   'id611',
   'id6',
   'id56']],
 'distances': [[0.3918856382369995,
   0.3959372937679291,
   0.39612558484077454,
   0.3994092345237732,
   0.4008343815803528,
   0.40150177478790283,
   0.40164804458618164,
   0.40415099263191223,
   0.4056999683380127,
   0.4065854847431183]],
 'metadatas': [[{'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '保健・衛生業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '保健・衛生業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'}]],
 'embeddings': None,
 'documents': [['コンテナより荷卸しした製材をリフトにより倉庫内に

In [15]:
results = collection.query(
    query_texts=["虫刺され"],
    n_results=10
)
results

{'ids': [['id431',
   'id49',
   'id547',
   'id679',
   'id782',
   'id631',
   'id377',
   'id374',
   'id407',
   'id304']],
 'distances': [[0.2651117742061615,
   0.2969682216644287,
   0.29818081855773926,
   0.2985743582248688,
   0.2991569936275482,
   0.2995659410953522,
   0.3005407154560089,
   0.3006022572517395,
   0.3009767532348633,
   0.3017663359642029]],
 'metadatas': [[{'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '接客娯楽業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '接客娯楽業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '清掃・と畜業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '接客娯楽業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'}]],
 'embeddings': None,
 'documents': [['調理場内洗い場にて、洗いものをしていた際、シンクの中

In [17]:
results = collection.query(
    query_texts=["不注意"],
    n_results=5
)
results

{'ids': [['id934', 'id167', 'id53', 'id395', 'id394']],
 'distances': [[0.40046584606170654,
   0.4175884425640106,
   0.4180261492729187,
   0.4215961992740631,
   0.4230976998806]],
 'metadatas': [[{'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '金融・広告業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': 'その他の事業'}]],
 'embeddings': None,
 'documents': [['資材整理中、周辺が暗かった為、地面の凸凹に気付かず、躓き、転倒。手首を強打し、負傷した。',
   '自宅訪問中、対向車が車線をはずれ、衝突した。',
   '休憩室にて、作業服に着替える際、誤って転倒。足甲を骨折した。',
   '国道を走行中、右折レーンの最後尾の停止した車両と前方不注意により、追突した。',
   '国道を走行中、右折レーンの最後尾の停止した車両と前方不注意により、追突した。']],
 'uris': None,
 'data': None}

In [18]:
results = collection.query(
    query_texts=["交通事故"],
    n_results=5
)
results

{'ids': [['id333', 'id292', 'id79', 'id939', 'id173']],
 'distances': [[0.2983427941799164,
   0.2989140450954437,
   0.30171361565589905,
   0.3021944463253021,
   0.30408617854118347]],
 'metadatas': [[{'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'}]],
 'embeddings': None,
 'documents': [['製品を客先へ搬送中、交差点にて、赤信号のために停止していたところ、後方から来た車に追突され、衝撃により首を負傷した。',
   '残土搬出後現場に戻った際、車両２台で連なり走行中、前方の車両が路側に停車したところ、後方の車両が追突。前方の車両の運転手は怪我はなかったが、後方の車両を運転していた被災者は、大腿骨を骨折した。',
   '市交差点にて、赤信号停止中、後方から走行して来た車両に追突された。',
   'タクシーで走行中、交差点の信号で停止した際、後方から追突された。',
   '客車の運転代行中、道路の確認の為、停車したところ、後続の随伴車両に追突された。']],
 'uris': None,
 'data': None}